In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
%matplotlib inline
from tasks import wsd
import pandas as pd
from tqdm.auto import tqdm
from sklearn.metrics import classification_report
from utils.dataset_download import harvest_data_from_extended_senses
from utils.classificaton_utils import binarize, vectorize_target_expressions,cosine_similiarity
from sklearn.svm import LinearSVC
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier


In [6]:
tqdm.pandas()
lemma,pos = 'machine', "NN"
senses = {'machine_nn01-38474140'} # machine_nn01-38475772 machine_nn01-38475923 machine_nn01-38475835 machine_nn01-38474140
relations = ['seed','synonym'] # ,'descendant','sibling'
eval_mode = "lemma_etal" # lemma or lemma_etal
    

In [11]:
df_train, df_val, df_test = binarize(lemma,
                        pos,
                        senses, 
                        relations,
                        strict_filter=True,
                        start=1700,
                        end=2000,
                        filter_val_by_year=False,
                        filter_test_by_year=False,
                        eval_mode=eval_mode)

# senses before filtering by date = 517
# senses after filtering by date = 433


# of seed senses 26 
# of synonyms 383 
# of branch senses 0


# of seeds selected 1 
# of synonyms selected 44 
# of branches selected 0
[LOG] 3560 quotations selected
[LOG] train = 2136 val = 534 test = 890 quotations


In [15]:

vector_col = "vector_bert_base_-1,-2,-3,-4_mean"

In [16]:

df = pd.concat([df_train,df_val], axis=0)
X,y = list(df[vector_col].values), list(df.label.values)

In [17]:
svm_model = LinearSVC(random_state=0, C=.1, tol=1e-5,class_weight='balanced')
svm_model.fit(X,y)
y_pred = wsd.clf_svm(vector_col,df_test, svm_model)
print(classification_report(y_pred,list(df_test.label.values)))

              precision    recall  f1-score   support

           0       0.94      0.95      0.94       808
           1       0.44      0.38      0.41        82

    accuracy                           0.90       890
   macro avg       0.69      0.66      0.67       890
weighted avg       0.89      0.90      0.89       890



In [20]:

perc_model = Perceptron(validation_fraction=.2, early_stopping=True,class_weight='balanced')
perc_model.fit(X,y)
y_pred = wsd.clf_perceptron(vector_col,df_test, perc_model)
print(classification_report(y_pred,list(df_test.label.values)))

              precision    recall  f1-score   support

           0       0.90      0.97      0.94       759
           1       0.70      0.38      0.50       131

    accuracy                           0.89       890
   macro avg       0.80      0.68      0.72       890
weighted avg       0.87      0.89      0.87       890



In [22]:
mlperc_model = MLPClassifier(validation_fraction=.2, early_stopping=True, solver='lbfgs',activation='relu')
mlperc_model.fit(X,y)
y_pred = wsd.clf_perceptron(vector_col,df_test, mlperc_model)
print(classification_report(y_pred,list(df_test.label.values)))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95       819
           1       0.44      0.44      0.44        71

    accuracy                           0.91       890
   macro avg       0.69      0.69      0.69       890
weighted avg       0.91      0.91      0.91       890

